In [15]:
import pandas as pd
import plotly.express as px

### Reading

In [16]:
time_df_ondemand = pd.read_csv('data/programs_time_results_owl.mp4_7_ondemand.csv')
time_df_nodemand = pd.read_csv('data/programs_time_results_owl.mp4_7_nodemand.csv')
time_df_ondemand['on_demand'] = True
time_df_nodemand['on_demand'] = False
time_df_ondemand.loc[(time_df_ondemand["p_version"] == 0) | (time_df_ondemand["p_version"] == 1), 'on_demand'] = False

In [17]:
time_df = pd.concat([time_df_ondemand, time_df_nodemand])

### Extracting data + Preprocessing

In [18]:
# take frame numbers
frame_numbers = time_df['frames_num'].iloc[0]
time_df # look

,iteration,n_workers,p_version,completion_time,frames_num,frames_motion_num,on_demand
0,1,1,0,83268667,251,160,False
1,1,1,1,82409166,251,160,False
2,1,1,2,163096483,251,160,True
3,1,1,3,153164577,251,160,True
4,1,2,1,41480267,251,160,False
...,...,...,...,...,...,...,...
965,10,31,2,6208564,251,160,False
966,10,31,3,10379300,251,160,False
967,10,32,1,6391960,251,160,False
968,10,32,2,6569208,251,160,False


In [19]:
# average across the different iterations
mean_time_df = time_df.groupby(['p_version', 'n_workers', 'on_demand'])['completion_time'].mean().reset_index()

# get the sequential time
sequential_time = mean_time_df[mean_time_df['p_version'] == 0]['completion_time'][0]
mean_time_df.drop(mean_time_df[mean_time_df['p_version'] == 0].index, inplace=True)

In [20]:
# add a speed up column
mean_time_df['speed_up'] = sequential_time / mean_time_df['completion_time']

In [21]:
# add ideal speed_up
for i in range(1,33):
    mean_time_df = mean_time_df.append({'p_version': 4, 'n_workers': i, 'speed_up':i}, ignore_index=True)

/var/folders/pg/z9tl77l52td7br14x19sr8n40000gn/T/ipykernel_3751/3191150293.py:3: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/pg/z9tl77l52td7br14x19sr8n40000gn/T/ipykernel_3751/3191150293.py:3: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/pg/z9tl77l52td7br14x19sr8n40000gn/T/ipykernel_3751/3191150293.py:3: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/pg/z9tl77l52td7br14x19sr8n40000gn/T/ipykernel_3751/3191150293.py:3: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/pg/z9tl77l52td7br14x19sr8n40000gn/T/ipykernel_3751/3191150293.py:3: FutureWarning:

The frame.append method is deprecated a

In [22]:
# top service time in milliseconds
program1_top_time = (mean_time_df[mean_time_df['p_version'] == 1].sort_values(by='completion_time')['completion_time'].iloc[0])/(frame_numbers) # millisecondi di service time
program2_top_time = (mean_time_df[mean_time_df['p_version'] == 2].sort_values(by='completion_time')['completion_time'].iloc[0])/(frame_numbers) # millisecondi di service time
program3_top_time = (mean_time_df[mean_time_df['p_version'] == 3].sort_values(by='completion_time')['completion_time'].iloc[0])/(frame_numbers) # millisecondi di service time

program1_top_time_nworkers = (mean_time_df[mean_time_df['p_version'] == 1].sort_values(by='completion_time')['n_workers'].iloc[0])
program2_top_time_nworkers = (mean_time_df[mean_time_df['p_version'] == 2].sort_values(by='completion_time')['n_workers'].iloc[0])
program3_top_time_nworkers = (mean_time_df[mean_time_df['p_version'] == 3].sort_values(by='completion_time')['n_workers'].iloc[0])

In [23]:
def define_readable_p_version(row):
    if row['p_version'] == 0:
        return "sequential"
    if row['p_version'] == 1:
        return "thread"
    if row['p_version'] == 2:
        if row['on_demand'] == 0:
            return "ff1"
        if row['on_demand'] == 1:
            return "ff1_demand"
    if row['p_version'] == 3:
        if row['on_demand'] == 0:
            return "ff2"
        if row['on_demand'] == 1:
            return "ff2_demand"
    if row['p_version'] == 4:
        return "ideal"

In [24]:
mean_time_df['p_version_readable'] = mean_time_df.apply(lambda row: define_readable_p_version(row), axis=1)

### Data analysis

#### Fast Flow comparison

In [25]:
fastflow_filter = (mean_time_df['p_version'] == 2) | (mean_time_df['p_version'] == 3) | (mean_time_df['p_version'] == 4)
fig = px.line(mean_time_df[fastflow_filter], x="n_workers", y="speed_up",color='p_version_readable', title="Speedup", line_dash='p_version_readable', ) 
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()

In [26]:
# top service time in milliseconds
ff1_top_time = (mean_time_df[mean_time_df['p_version_readable'] == "ff1"].sort_values(by='completion_time')['completion_time'].iloc[0])/(frame_numbers) # millisecondi di service time
ff1_demand_top_time = (mean_time_df[mean_time_df['p_version_readable'] == "ff1_demand"].sort_values(by='completion_time')['completion_time'].iloc[0])/(frame_numbers) # millisecondi di service time
ff2_top_time = (mean_time_df[mean_time_df['p_version_readable'] == "ff2"].sort_values(by='completion_time')['completion_time'].iloc[0])/(frame_numbers) # millisecondi di service time
ff2_demand_top_time = (mean_time_df[mean_time_df['p_version_readable'] == "ff2_demand"].sort_values(by='completion_time')['completion_time'].iloc[0])/(frame_numbers) # millisecondi di service time
thread_top_time = (mean_time_df[mean_time_df['p_version_readable'] == "thread"].sort_values(by='completion_time')['completion_time'].iloc[0])/(frame_numbers) # millisecondi di service time

# number of workers for the top times
ff1_top_time_nworkers = (mean_time_df[mean_time_df['p_version_readable'] == "ff1"].sort_values(by='completion_time')['n_workers'].iloc[0])
ff1_demand_top_time_nworkers = (mean_time_df[mean_time_df['p_version_readable'] == "ff1_demand"].sort_values(by='completion_time')['n_workers'].iloc[0])
ff2_top_time_nworkers = (mean_time_df[mean_time_df['p_version_readable'] == "ff2"].sort_values(by='completion_time')['n_workers'].iloc[0])
ff2_demand_top_time_nworkers = (mean_time_df[mean_time_df['p_version_readable'] == "ff2_demand"].sort_values(by='completion_time')['n_workers'].iloc[0])
thread_top_time_nworkers = (mean_time_df[mean_time_df['p_version_readable'] == "thread"].sort_values(by='completion_time')['n_workers'].iloc[0])

In [27]:
print("Thread top time = " + str(thread_top_time) + '. Using ' + str(thread_top_time_nworkers) + " workers")
print("FastFlow1 top time = " + str(ff1_top_time) + '. Using ' + str(ff1_top_time_nworkers) + " workers")
print("FastFlow1 on dem. sched. top time = " + str(ff1_demand_top_time) + '. Using ' + str(ff1_demand_top_time_nworkers) + " workers")
print("FastFlow2 top time = " + str(ff2_top_time) + '. Using ' + str(ff2_top_time_nworkers) + " workers")
print("FastFlow2 on dem. sched. top time = " + str(ff2_demand_top_time) + '. Using ' + str(ff2_demand_top_time_nworkers) + " workers")

Thread top time = 25391.893824701194. Using 32 workers
FastFlow1 top time = 28646.64501992032. Using 31 workers
FastFlow1 on dem. sched. top time = 24964.274501992033. Using 31 workers
FastFlow2 top time = 40941.489641434266. Using 18 workers
FastFlow2 on dem. sched. top time = 40651.46454183267. Using 22 workers


#### Fast Flow - Thread (comparison)

In [28]:
fastflow_filter = (mean_time_df['p_version_readable'] == "ff1_demand") | (mean_time_df['p_version'] == 1) | (mean_time_df['p_version'] == 4)
fig = px.line(mean_time_df[fastflow_filter], x="n_workers", y="speed_up",color='p_version_readable', title="Speedup", line_dash='p_version_readable', ) 
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()